In [ ]:
# import Libraries
import yt_dlp
from pydub import AudioSegment
from src.youtube_downloader import YouTubeDownloader
# pip install yt-dlp
# pip install pydub
# pip install moviepy

In [ ]:
# pip install --upgrade yt-dlp

# Initialize downloader
downloader = YouTubeDownloader()

# If you need to specify ffmpeg path:
# downloader = YouTubeDownloader(ffmpeg_path='/usr/local/bin/ffmpeg')

In [ ]:
from yt_dlp import YoutubeDL

def download_youtube_video(url):
    # Configuración para yt-dlp
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'merge_output_format': 'mp4',
        'cookiesfrombrowser': ('chrome',),
        'quiet': False,
        'ffmpeg_location': '/usr/local/bin/ffmpeg',
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            print(f"Descargando: {url}")
            ydl.download([url])
            print("Descarga completada.")
    except Exception as e:
        print(f"Error al descargar el video: {e}")

# Example: Download a video
url = input('Enter YouTube URL: ')
download_youtube_video(url)

Introduce la URL de YouTube:  https://youtu.be/RKZ3gZqfIlQ


Descargando: https://youtu.be/RKZ3gZqfIlQ
Extracting cookies from chrome
Extracted 3372 cookies from chrome
[youtube] Extracting URL: https://youtu.be/RKZ3gZqfIlQ
[youtube] RKZ3gZqfIlQ: Downloading webpage
[youtube] RKZ3gZqfIlQ: Downloading tv client config
[youtube] RKZ3gZqfIlQ: Downloading player 6450230e
[youtube] RKZ3gZqfIlQ: Downloading tv player API JSON
[info] RKZ3gZqfIlQ: Downloading 1 format(s): 136+251
[download] Destination: COMO TE OLVIDO EN ACORDEÓN (TUTORIAL COMPLETO, CON SU INICIO, ACOMPAÑAMIENTO E INTERMEDIO).f136.mp4
[download] 100% of  385.31MiB in 00:00:34 at 11.30MiB/s    
[download] Destination: COMO TE OLVIDO EN ACORDEÓN (TUTORIAL COMPLETO, CON SU INICIO, ACOMPAÑAMIENTO E INTERMEDIO).f251.webm
[download] 100% of   26.73MiB in 00:00:02 at 10.87MiB/s    
[Merger] Merging formats into "COMO TE OLVIDO EN ACORDEÓN (TUTORIAL COMPLETO, CON SU INICIO, ACOMPAÑAMIENTO E INTERMEDIO).mp4"
Deleting original file COMO TE OLVIDO EN ACORDEÓN (TUTORIAL COMPLETO, CON SU INICIO, ACO

In [ ]:
# Audio converter from mkv to mp3

from pydub import AudioSegment
import os

# Manually set the ffmpeg and ffprobe paths
os.environ["PATH"] += ":/usr/local/bin"  # Add ffmpeg location to PATH
AudioSegment.ffmpeg = "/usr/local/bin/ffmpeg"
AudioSegment.ffprobe = "/usr/local/bin/ffprobe"

# Check if the paths are being set correctly
from pydub.utils import which
print("ffmpeg path:", which("ffmpeg"))
print("ffprobe path:", which("ffprobe"))

# Path to your MKV file
input_file = "Plank_Workout.mkv"
output_file = "Plank_Workout.mp3"

try:
    # Load the MKV file
    audio = AudioSegment.from_file(input_file)

    # Convert to MP3
    audio.export(output_file, format="mp3", bitrate="192k")
    print(f"Audio converted successfully to {output_file}")
except Exception as e:
    print(f"An error occurred during conversion: {e}")

# Example: Convert video to MP3
input_file = input('Enter video filename to convert: ')
downloader.convert_to_mp3(input_file)

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


ffmpeg path: /usr/local/bin/ffmpeg
ffprobe path: /usr/local/bin/ffprobe
Audio converted successfully to Plank_Workout.mp3


In [ ]:
# Audio converter mp3 to midi

# Example: Download and convert in one step
url = input('Enter YouTube URL: ')
downloader.process_url(url, convert_to_audio=True, delete_video=False)


